<a href="https://colab.research.google.com/github/liligould/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.2 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-09 20:33:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-07-09 20:33:38 (10.5 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

In [22]:
# Create new df and retrieve rows where total_votes is greater than or equal to 20
vine_20_df = vine_df.filter("total_votes>=20")
vine_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [25]:
# Create new df and retrieve rows where number of helpful_votes divided by total_votes is greater than or equal than 50%
helpful_50_df = vine_df.filter("helpful_votes/total_votes>=0.5")
helpful_50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|                Y|
|  REFRE1LEKLAF|          4|            1|          1|   N|                Y|
|R25Z4ADMYBHR66|          4|            1|          2|   N|                Y|
|R2SGL2ZAPGJH9I|          5|            1|          1|   N|                N|
| RZR1QCHTL8Q4Z|          5|            1|          1|   N|                N|
|R2M8ZRIWBPQGRR|          4|            1|          1|   N|                Y|
|R3C2J53J8UDDPW|          5|            5|          5|   N|                N|
|R37RFLBIGKHA9L|          4|            1|          1|   N|                N|
|R39NGBRCQYAJRH|          4|            4|          4|   N|     

In [35]:
# filter helpful_50_df and retrieve rows where review was written as part of the vine program 'Y'
vine_paid_df = helpful_50_df.filter("vine == 'Y'")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2RZF08POZS24Y|          3|            1|          1|   Y|                N|
| RL3APATHQK2NS|          4|            1|          1|   Y|                N|
| RUYAP1ZCYRED7|          3|            1|          2|   Y|                N|
|R1SWZLB1A4460R|          2|            1|          2|   Y|                N|
| RJUAWYTKIM4V7|          3|            1|          1|   Y|                N|
|R3NP8J9PAV58Z2|          4|            2|          2|   Y|                N|
|R3BQZZOY6KJXRR|          5|            1|          2|   Y|                N|
|R3EZBSCDWLTTGO|          4|            1|          1|   Y|                N|
| RMTJWZKUPN18Z|          4|            1|          1|   Y|                N|
|R3105EK6O6JYDY|          3|            1|          1|   Y|     

In [36]:
# filter helpful_50_df and retrieve rows where review was written as part of the vine program 'N'
vine_unpaid_df = helpful_50_df.filter("vine == 'N'")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|                Y|
|  REFRE1LEKLAF|          4|            1|          1|   N|                Y|
|R25Z4ADMYBHR66|          4|            1|          2|   N|                Y|
|R2SGL2ZAPGJH9I|          5|            1|          1|   N|                N|
| RZR1QCHTL8Q4Z|          5|            1|          1|   N|                N|
|R2M8ZRIWBPQGRR|          4|            1|          1|   N|                Y|
|R3C2J53J8UDDPW|          5|            5|          5|   N|                N|
|R37RFLBIGKHA9L|          4|            1|          1|   N|                N|
|R39NGBRCQYAJRH|          4|            4|          4|   N|     

In [38]:
# determine number of reviews paid 
total_reviews_paid = vine_paid_df.count()
total_reviews_paid

709

In [39]:
#unpaid
total_reviews_unpaid = vine_unpaid_df.count()
total_reviews_unpaid

2386061

In [41]:
# number of 5 star reviews paid
star_reviews_paid = vine_paid_df.filter("star_rating == 5").count()
star_reviews_paid

159

In [42]:
# unpaid
star_reviews_unpaid = vine_unpaid_df.filter("star_rating == 5").count()
star_reviews_unpaid

1676546

In [45]:
# percentage of 5 star reviews paid
paid_reviews_percentage = star_reviews_paid/total_reviews_paid * 100
paid_reviews_percentage

22.425952045133993

In [46]:
# unpaid
unpaid_reviews_percentage = star_reviews_unpaid/total_reviews_unpaid * 100
unpaid_reviews_percentage

70.26417178772881